i22-1653                     Insharah Aman                                                                                                                                       


i22-1677                     Sajer Tasleem                                                                                                                                       


i22-7425                     Abdul Munim                                                                                                                                         



Cy-A
Infosec

# Shamir's Secret Sharing

**Shamir's Secret Sharing for Two-Person**

In [ ]:
import random

# function to compute modular inverse using the extended euclidean algorithm
def mod_inverse(a, p):
    """
    returns the modular inverse of a under modulo p using the extended euclidean algorithm.
    if the inverse exists, return the inverse; otherwise, raise an error.
    """
    t, new_t = 0, 1
    r, new_r = p, a
    while new_r != 0:
        quotient = r // new_r
        t, new_t = new_t, t - quotient * new_t
        r, new_r = new_r, r - quotient * new_r
    if r > 1:
        raise ValueError(f"modular inverse does not exist for {a} under modulo {p}")
    if t < 0:
        t = t + p
    return t

# function to generate the shares for shamir's secret sharing
def shamir_secret_sharing(secret, n, t, p):
    """
    secret: the secret to be shared.
    n: the total number of participants.
    t: the threshold (minimum number of participants required to reconstruct the secret).
    p: a prime number (modulus for the operations).
    """
    # step 1: randomly generate a polynomial of degree t-1
    coefficients = [secret] + [random.randint(0, p-1) for _ in range(t-1)]

    print("\ngenerated polynomial coefficients:")
    print(f"coefficients: {coefficients}")
    print("the polynomial is of the form:")
    print(f"f(x) = {coefficients[0]} + {coefficients[1]}x + ...")

    # step 2: generate the shares
    shares = []
    for i in range(1, n+1):
        # for each participant, calculate the y-value by evaluating the polynomial at x = i
        x = i
        y = sum([coefficients[j] * (x ** j) for j in range(t)]) % p
        shares.append((x, y))

    print("\ngenerated shares:")
    for i, (x, y) in enumerate(shares):
        print(f"participant {i+1}: share = ({x}, {y})")

    return shares, coefficients

# function to reconstruct the secret from a set of shares using lagrange interpolation
def reconstruct_secret(shares, p):
    """
    shares: a list of shares (x, y) used for reconstruction.
    p: a prime number (modulus for the operations).
    """
    print("\nreconstructing the secret using lagrange interpolation:")
    secret = 0
    n = len(shares)
    
    for i in range(n):
        xi, yi = shares[i]
        print(f"  - share {i+1}: ({xi}, {yi})")
        
        # calculate the lagrange basis polynomial li
        li = 1
        for j in range(n):
            if i != j:
                xj, _ = shares[j]
                print(f"    - calculating term for j={j+1} (xj={xj})")
                
                # numerator = x - xj (we evaluate at x=0, so numerator becomes -xj)
                numerator = (-xj) % p
                
                # denominator = xi - xj
                denominator = (xi - xj) % p
                
                # term = numerator / denominator
                inv_denominator = mod_inverse(denominator, p)
                term = (numerator * inv_denominator) % p
                
                li = (li * term) % p
                print(f"    - current term: {term}, li: {li}")
        
        # add the contribution of the current share to the secret
        secret = (secret + yi * li) % p
        print(f"    - partial secret after share {i+1}: {secret} (mod {p})")
    
    return secret


# function to test shamir's secret sharing scheme with user input and display iterations
def test_shamir_secret_sharing():
    print("\n--- two-person key sharing ---")
    
    # ensure the number of participants is fixed to 2 for key sharing
    n = 2
    t = 2  # threshold for reconstruction
    
    # take user input for the secret and prime modulus
    secret = int(input("enter the secret to be shared: "))
    p = int(input("enter a prime number (modulus for the operations): "))

    print("\n--- generating shares ---")
    # step 1: generate shares
    shares, coefficients = shamir_secret_sharing(secret, n, t, p)

    # step 2: select both shares (since n = 2 and t = 2, all shares are required for reconstruction)
    selected_shares = shares[:t]
    print(f"\nselected shares for reconstruction: {selected_shares}")

    print("\n--- reconstructing the secret ---")
    # step 3: reconstruct the secret
    reconstructed_secret = reconstruct_secret(selected_shares, p)

    # step 4: check if the reconstruction is correct
    print(f"\nreconstructed secret: {reconstructed_secret}")
    if secret == reconstructed_secret:
        print("\nsecret reconstructed successfully!")
    else:
        print(f"error: the reconstructed secret {reconstructed_secret} does not match the original secret {secret}!")


test_shamir_secret_sharing()



--- two-person key sharing ---

--- generating shares ---

generated polynomial coefficients:
coefficients: [42, 6]
the polynomial is of the form:
f(x) = 42 + 6x + ...

generated shares:
participant 1: share = (1, 48)
participant 2: share = (2, 54)

selected shares for reconstruction: [(1, 48), (2, 54)]

--- reconstructing the secret ---

reconstructing the secret using lagrange interpolation:
  - share 1: (1, 48)
    - calculating term for j=2 (xj=2)
    - current term: 2, li: 2
    - partial secret after share 1: 96 (mod 211)
  - share 2: (2, 54)
    - calculating term for j=1 (xj=1)
    - current term: 210, li: 210
    - partial secret after share 2: 42 (mod 211)

reconstructed secret: 42

secret reconstructed successfully!


 **Shamir's Secret Sharing for n-person**


In [5]:
import random

# function to compute modular inverse using the extended euclidean algorithm
def mod_inverse(a, p):
    """
    returns the modular inverse of a under modulo p using the extended euclidean algorithm.
    if the inverse exists, return the inverse; otherwise, raise an error.
    """
    t, new_t = 0, 1
    r, new_r = p, a
    while new_r != 0:
        quotient = r // new_r
        t, new_t = new_t, t - quotient * new_t
        r, new_r = new_r, r - quotient * new_r
    if r > 1:
        raise ValueError(f"modular inverse does not exist for {a} under modulo {p}")
    if t < 0:
        t = t + p
    return t

# function to check if the number is prime
def is_prime(n):
    """returns true if n is a prime number, otherwise false"""
    if n <= 1:
        return False
    for i in range(2, int(n**0.5) + 1):
        if n % i == 0:
            return False
    return True

# function to generate the shares for shamir's secret sharing
def shamir_secret_sharing(secret, n, t, p):
    """
    secret: the secret to be shared.
    n: the total number of participants.
    t: the threshold (minimum number of participants required to reconstruct the secret).
    p: a prime number (modulus for the operations).
    """
    # step 1: randomly generate a polynomial of degree t-1
    coefficients = [secret] + [random.randint(0, p-1) for _ in range(t-1)]

    print("\ngenerated polynomial coefficients:")
    print(f"coefficients: {coefficients}")
    print("the polynomial is of the form: f(x) = ", end="")
    for i, coef in enumerate(coefficients):
        if i == len(coefficients) - 1:
            print(f"{coef}")
        else:
            print(f"{coef} + ", end="")
    
    # step 2: generate the shares
    shares = []
    for i in range(1, n + 1):
        x = i
        y = sum([coefficients[j] * (x ** j) for j in range(t)]) % p
        shares.append((x, y))

    print("\ngenerated shares:")
    for i, (x, y) in enumerate(shares):
        print(f"participant {i+1}: share = ({x}, {y})")

    return shares, coefficients

# function to reconstruct the secret from a set of shares using lagrange interpolation
def reconstruct_secret(shares, p):
    """
    shares: a list of shares (x, y) used for reconstruction.
    p: a prime number (modulus for the operations).
    """
    print("\nreconstructing the secret using lagrange interpolation:")
    secret = 0
    n = len(shares)
    
    for i in range(n):
        xi, yi = shares[i]
        print(f"  - share {i+1}: ({xi}, {yi})")
        
        # calculate the lagrange basis polynomial li
        li = 1
        for j in range(n):
            if i != j:
                xj, _ = shares[j]
                print(f"    - calculating term for j={j+1} (xj={xj})")
                
                # numerator = x - xj (we evaluate at x=0, so numerator becomes -xj)
                numerator = (-xj) % p
                
                # denominator = xi - xj
                denominator = (xi - xj) % p
                
                # term = numerator / denominator
                inv_denominator = mod_inverse(denominator, p)
                term = (numerator * inv_denominator) % p
                
                li = (li * term) % p
                print(f"    - current term: {term}, li: {li}")
        
        # add the contribution of the current share to the secret
        secret = (secret + yi * li) % p
        print(f"    - partial secret after share {i+1}: {secret} (mod {p})")
    
    return secret

# function to test shamir's secret sharing scheme with user input and display iterations
def test_shamir_secret_sharing():
    print("\n--- shamir's secret sharing ---")
    
    # ensure the number of participants is input by the user
    n = int(input("enter the total number of participants: "))
    
    # set the threshold (minimum number of participants required to reconstruct the secret) to 2
    t = 2
    
    # ensure a valid prime modulus is entered
    while True:
        p = int(input("enter a prime number (modulus for the operations): "))
        if is_prime(p):
            break
        else:
            print(f"{p} is not a prime number. please enter a prime number.")
    
    # take user input for the secret
    secret = int(input("enter the secret to be shared: "))

    print("\n--- generating shares ---")
    # step 1: generate shares
    shares, coefficients = shamir_secret_sharing(secret, n, t, p)

    # step 2: select shares for reconstruction (we need at least 't' shares)
    selected_shares = shares[:t]
    print(f"\nselected shares for reconstruction: {selected_shares}")

    print("\n--- reconstructing the secret ---")
    # step 3: reconstruct the secret
    reconstructed_secret = reconstruct_secret(selected_shares, p)

    # step 4: check if the reconstruction is correct
    print(f"\nreconstructed secret: {reconstructed_secret}")
    if secret == reconstructed_secret:
        print("\nsecret reconstructed successfully!")
    else:
        print(f"error: the reconstructed secret {reconstructed_secret} does not match the original secret {secret}!")


test_shamir_secret_sharing()



--- shamir's secret sharing ---

--- generating shares ---

generated polynomial coefficients:
coefficients: [42, 208]
the polynomial is of the form: f(x) = 42 + 208

generated shares:
participant 1: share = (1, 39)
participant 2: share = (2, 36)

selected shares for reconstruction: [(1, 39), (2, 36)]

--- reconstructing the secret ---

reconstructing the secret using lagrange interpolation:
  - share 1: (1, 39)
    - calculating term for j=2 (xj=2)
    - current term: 2, li: 2
    - partial secret after share 1: 78 (mod 211)
  - share 2: (2, 36)
    - calculating term for j=1 (xj=1)
    - current term: 210, li: 210
    - partial secret after share 2: 42 (mod 211)

reconstructed secret: 42

secret reconstructed successfully!


# GUI IMPLEMENTATION

**Shamir's Secret Sharing for Two-Person**

In [8]:
import pygame
import random
import sys
from pygame.locals import *

# function to compute modular inverse using the extended euclidean algorithm
def mod_inverse(a, p):
    """
    returns the modular inverse of a under modulo p using the extended euclidean algorithm.
    if the inverse exists, return the inverse; otherwise, raise an error.
    """
    t, new_t = 0, 1
    r, new_r = p, a
    steps = []
    steps.append(f"computing modular inverse of {a} under modulo {p}:")
    while new_r != 0:
        quotient = r // new_r
        steps.append(f"  {r} = {quotient} × {new_r} + {r - quotient * new_r}")
        t, new_t = new_t, t - quotient * new_t
        r, new_r = new_r, r - quotient * new_r
    if r > 1:
        steps.append(f"inverse doesn't exist for {a} under modulo {p}")
        raise ValueError(f"modular inverse does not exist for {a} under modulo {p}")
    if t < 0:
        t = t + p
        steps.append(f"adjusting negative inverse: {t-p} → {t}")
    steps.append(f"final inverse: {t}")
    return t, steps

# function to generate shares with detailed steps
def shamir_secret_sharing_with_steps(secret, n, t, p):
    steps = []
    steps.append(f"generating {n} shares with threshold {t} using prime modulus {p}:")
    
    # step 1: generate polynomial coefficients
    coefficients = [secret] + [random.randint(0, p-1) for _ in range(t-1)]
    steps.append("\ngenerated polynomial coefficients:")
    poly_str = "f(x) = " + " + ".join([f"{coef}x^{i}" for i, coef in enumerate(coefficients)])
    steps.append(poly_str)
    
    # step 2: generate shares
    shares = []
    steps.append("\ngenerating shares:")
    for i in range(1, n+1):
        x = i
        steps.append(f"\ncalculating share for participant {i} (x = {x}):")
        y = 0
        for j in range(t):
            term = coefficients[j] * (x ** j)
            y = (y + term) % p
            steps.append(f"  add term {j}: {coefficients[j]} × {x}^{j} = {term} → current sum: {y % p}")
        shares.append((x, y))
        steps.append(f"final y-value for participant {i}: ({x}, {y})")
    
    return shares, coefficients, steps

# function to reconstruct secret with detailed steps
def reconstruct_secret_with_steps(shares, p):
    steps = []
    steps.append(f"reconstructing secret from {len(shares)} shares using modulus {p}:")
    secret = 0
    n = len(shares)
    
    for i in range(n):
        xi, yi = shares[i]
        steps.append(f"\nprocessing share {i+1}: ({xi}, {yi})")
        
        li = 1
        steps.append(f"  calculating lagrange basis polynomial l_{i+1}(0):")
        
        for j in range(n):
            if i != j:
                xj, _ = shares[j]
                steps.append(f"    term for share {j+1} (x={xj}):")
                
                numerator = (-xj) % p
                denominator = (xi - xj) % p
                steps.append(f"      numerator (0 - x{j+1}): {-xj} mod {p} = {numerator}")
                steps.append(f"      denominator (x{i+1} - x{j+1}): {xi-xj} mod {p} = {denominator}")
                
                try:
                    inv_denominator, inv_steps = mod_inverse(denominator, p)
                    steps.extend(["      " + s for s in inv_steps])
                    
                    term = (numerator * inv_denominator) % p
                    steps.append(f"      term value: {numerator} × {inv_denominator} mod {p} = {term}")
                    
                    li = (li * term) % p
                    steps.append(f"      current l_{i+1}(0): {li}")
                except ValueError as e:
                    steps.append(f"      error: {e}")
                    return None, steps
        
        contribution = (yi * li) % p
        steps.append(f"  contribution of share {i+1}: y_{i+1} × l_{i+1}(0) = {yi} × {li} mod {p} = {contribution}")
        
        secret = (secret + contribution) % p
        steps.append(f"  partial secret after share {i+1}: {secret}")
    
    steps.append(f"\nfinal reconstructed secret: {secret}")
    return secret, steps

# initialize pygame
pygame.init()
WIDTH, HEIGHT = 1100, 850
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Two-Person Key Sharing with Detailed Steps")

# colors
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
GRAY = (200, 200, 200)
BLUE = (0, 0, 255)
RED = (255, 0, 0)
GREEN = (0, 255, 0)
LIGHT_BLUE = (173, 216, 230)
LIGHT_GREEN = (144, 238, 144)

# fonts
title_font = pygame.font.SysFont('Arial', 32, bold=True)
font = pygame.font.SysFont('Arial', 20)
small_font = pygame.font.SysFont('Arial', 16)

# input boxes
input_boxes = [
    {"rect": pygame.Rect(350, 150, 200, 40), "text": "", "active": False, "label": "Secret:"},
    {"rect": pygame.Rect(350, 250, 200, 40), "text": "", "active": False, "label": "Prime p:"}
]

# buttons
generate_button = pygame.Rect(350, 350, 200, 50)
reconstruct_button = pygame.Rect(350, 450, 200, 50)

# state variables
shares = []
coefficients = []
secret = None
p = None
reconstructed_secret = None
share_generation_steps = []
reconstruction_steps = []
current_tab = "generation"  # "generation" or "reconstruction"
scroll_offset_gen = 0
scroll_offset_rec = 0

def draw_text(text, font, color, x, y):
    text_surface = font.render(text, True, color)
    screen.blit(text_surface, (x, y))

def draw_multiline_text(lines, font, color, x, y, line_height=25, max_lines=15):
    visible_lines = lines[scroll_offset_gen if current_tab == "generation" else scroll_offset_rec:]
    visible_lines = visible_lines[:max_lines]
    for i, line in enumerate(visible_lines):
        draw_text(line, font, color, x, y + i * line_height)

def main():
    global secret, p, shares, coefficients, reconstructed_secret
    global share_generation_steps, reconstruction_steps, current_tab
    global scroll_offset_gen, scroll_offset_rec
    
    running = True
    while running:
        screen.fill(WHITE)
        
        # draw title
        draw_text("Two-Person Key Sharing with Detailed Steps", title_font, BLACK, WIDTH//2 - 300, 30)
        
        # draw input boxes
        for box in input_boxes:
            color = BLUE if box["active"] else BLACK
            pygame.draw.rect(screen, GRAY, box["rect"])
            pygame.draw.rect(screen, color, box["rect"], 2)
            
            # draw label
            draw_text(box["label"], font, BLACK, box["rect"].x - 100, box["rect"].y + 5)
            
            # draw input text
            text_surface = font.render(box["text"], True, BLACK)
            screen.blit(text_surface, (box["rect"].x + 5, box["rect"].y + 5))
        
        # draw buttons
        pygame.draw.rect(screen, GREEN, generate_button)
        draw_text("Generate Shares", font, BLACK, generate_button.x + 30, generate_button.y + 15)
        
        pygame.draw.rect(screen, RED, reconstruct_button)
        draw_text("Reconstruct Secret", font, BLACK, reconstruct_button.x + 15, reconstruct_button.y + 15)
        
        # tab selection
        pygame.draw.rect(screen, LIGHT_GREEN if current_tab == "generation" else GRAY, (50, 520, 200, 40))
        pygame.draw.rect(screen, LIGHT_BLUE if current_tab == "reconstruction" else GRAY, (300, 520, 250, 40))
        draw_text("Generation Steps", font, BLACK, 80, 530)
        draw_text("Reconstruction Steps", font, BLACK, 330, 530)
        
        # display steps based on current tab
        steps_area = pygame.Rect(50, 570, WIDTH - 100, 250)
        pygame.draw.rect(screen, WHITE, steps_area)
        pygame.draw.rect(screen, BLACK, steps_area, 2)
        
        if current_tab == "generation" and share_generation_steps:
            draw_multiline_text(share_generation_steps, small_font, BLACK, steps_area.x + 10, steps_area.y + 10, 20)
            if len(share_generation_steps) > 15:
                draw_text("↑ Scroll Up ↑", small_font, BLACK, WIDTH//2 - 50, steps_area.y - 30)
                draw_text("↓ Scroll Down ↓", small_font, BLACK, WIDTH//2 - 50, steps_area.y + 260)
        elif current_tab == "reconstruction" and reconstruction_steps:
            draw_multiline_text(reconstruction_steps, small_font, BLACK, steps_area.x + 10, steps_area.y + 10, 20)
            if len(reconstruction_steps) > 15:
                draw_text("↑ Scroll Up ↑", small_font, BLACK, WIDTH//2 - 50, steps_area.y - 30)
                draw_text("↓ Scroll Down ↓", small_font, BLACK, WIDTH//2 - 50, steps_area.y + 260)
        
        # display final result
        if reconstructed_secret is not None:
            result_y = 830
            if reconstructed_secret == secret:
                draw_text(f"Success! Reconstructed secret: {reconstructed_secret}", font, GREEN, 50, result_y)
            else:
                draw_text(f"Error! Reconstructed: {reconstructed_secret}, Original: {secret}", font, RED, 50, result_y)
        
        for event in pygame.event.get():
            if event.type == QUIT:
                running = False
            
            if event.type == MOUSEBUTTONDOWN:
                # check input box clicks
                for box in input_boxes:
                    if box["rect"].collidepoint(event.pos):
                        box["active"] = True
                    else:
                        box["active"] = False
                
                # check generate button click
                if generate_button.collidepoint(event.pos):
                    try:
                        secret = int(input_boxes[0]["text"])
                        p = int(input_boxes[1]["text"])
                        shares, coefficients, share_generation_steps = shamir_secret_sharing_with_steps(secret, 2, 2, p)
                        reconstruction_steps = []
                        reconstructed_secret = None
                        current_tab = "generation"
                        scroll_offset_gen = 0
                    except ValueError:
                        print("Invalid input")
                
                # check reconstruct button click
                if reconstruct_button.collidepoint(event.pos) and shares:
                    try:
                        reconstructed_secret, reconstruction_steps = reconstruct_secret_with_steps(shares, p)
                        current_tab = "reconstruction"
                        scroll_offset_rec = 0
                    except ValueError as e:
                        print(e)
                
                # check tab selection
                if pygame.Rect(50, 520, 200, 40).collidepoint(event.pos):
                    current_tab = "generation"
                elif pygame.Rect(300, 520, 250, 40).collidepoint(event.pos):
                    current_tab = "reconstruction"
            
            # handle scrolling
            if event.type == MOUSEWHEEL:
                if event.y > 0 and scroll_offset_gen > 0:  # scroll up
                    scroll_offset_gen -= 1
                elif event.y < 0 and scroll_offset_gen < len(share_generation_steps) - 15:  # scroll down
                    scroll_offset_gen += 1
            
            if event.type == KEYDOWN:
                # handle text input
                active_box = next((box for box in input_boxes if box["active"]), None)
                if active_box:
                    if event.key == K_RETURN:
                        active_box["active"] = False
                    elif event.key == K_BACKSPACE:
                        active_box["text"] = active_box["text"][:-1]
                    else:
                        if event.unicode.isdigit():
                            active_box["text"] += event.unicode
        
        pygame.display.flip()
    
    pygame.quit()
    sys.exit()

if __name__ == "__main__":
    main()


SystemExit: 

 **Shamir's Secret Sharing for n-person**


In [ ]:
import pygame
import random
import sys
from pygame.locals import *

# Function to compute modular inverse with steps
def mod_inverse_with_steps(a, p):
    steps = []
    steps.append(f"Computing modular inverse of {a} under modulo {p}:")
    t, new_t = 0, 1
    r, new_r = p, a
    while new_r != 0:
        quotient = r // new_r
        steps.append(f"  {r} = {quotient} × {new_r} + {r - quotient * new_r}")
        t, new_t = new_t, t - quotient * new_t
        r, new_r = new_r, r - quotient * new_r
    if r > 1:
        steps.append(f"Inverse doesn't exist for {a} under modulo {p}")
        raise ValueError(f"Modular inverse does not exist for {a} under modulo {p}")
    if t < 0:
        steps.append(f"Adjusting negative inverse: {t} → {t + p}")
        t = t + p
    steps.append(f"Final inverse: {t}")
    return t, steps

# Function to generate shares with detailed steps
def generate_shares_with_steps(secret, n, t, p):
    steps = []
    steps.append(f"Generating {n} shares with threshold {t} using prime modulus {p}:")

    # Generate polynomial coefficients
    coefficients = [secret] + [random.randint(0, p-1) for _ in range(t-1)]
    steps.append("\nGenerated polynomial coefficients:")
    poly_str = "f(x) = " + " + ".join([f"{coef}x^{i}" for i, coef in enumerate(coefficients)])
    steps.append(poly_str)

    # Generate shares
    shares = []
    steps.append("\nGenerating shares:")
    for i in range(1, n+1):
        x = i
        steps.append(f"\nCalculating share for participant {i} (x = {x}):")
        y = 0
        for j in range(t):
            term = coefficients[j] * (x ** j)
            y = (y + term) % p
            steps.append(f"  Add term {j}: {coefficients[j]} × {x}^{j} = {term} → Current sum: {y % p}")
        shares.append((x, y))
        steps.append(f"Final y-value for participant {i}: ({x}, {y})")
    
    return shares, coefficients, steps

# Function to reconstruct secret with detailed steps
def reconstruct_secret_with_steps(shares, p, threshold):
    steps = []
    steps.append(f"Reconstructing secret from {len(shares)} shares (threshold = {threshold}) using modulus {p}:")
    secret = 0
    
    # Use only the first 'threshold' shares for reconstruction
    reconstruction_shares = shares[:threshold]
    
    for i, (xi, yi) in enumerate(reconstruction_shares):
        steps.append(f"\nProcessing Share {i+1}: ({xi}, {yi})")
        
        li = 1
        steps.append(f"  Calculating Lagrange basis polynomial l_{i+1}(0):")
        
        for j, (xj, _) in enumerate(reconstruction_shares):
            if i != j:
                steps.append(f"    Term for Share {j+1} (x={xj}):")
                
                numerator = (-xj) % p
                denominator = (xi - xj) % p
                steps.append(f"      Numerator (0 - x{j+1}): {-xj} mod {p} = {numerator}")
                steps.append(f"      Denominator (x{i+1} - x{j+1}): {xi-xj} mod {p} = {denominator}")
                
                try:
                    inv_denominator, inv_steps = mod_inverse_with_steps(denominator, p)
                    steps.extend(["      " + s for s in inv_steps])
                    
                    term = (numerator * inv_denominator) % p
                    steps.append(f"      Term value: {numerator} × {inv_denominator} mod {p} = {term}")
                    
                    li = (li * term) % p
                    steps.append(f"      Current l_{i+1}(0): {li}")
                except ValueError as e:
                    steps.append(f"      Error: {e}")
                    return None, steps
        
        contribution = (yi * li) % p
        steps.append(f"  Contribution of Share {i+1}: y_{i+1} × l_{i+1}(0) = {yi} × {li} mod {p} = {contribution}")
        
        secret = (secret + contribution) % p
        steps.append(f"  Partial secret after Share {i+1}: {secret}")
    
    steps.append(f"\nFinal reconstructed secret: {secret}")
    return secret, steps

# GUI Application Class
class ShamirSecretSharingGUI:
    def __init__(self):
        pygame.init()
        self.WIDTH, self.HEIGHT = 1100, 850
        self.screen = pygame.display.set_mode((self.WIDTH, self.HEIGHT))
        pygame.display.set_caption("Generic Shamir's Secret Sharing")
        
        # Colors
        self.WHITE = (255, 255, 255)
        self.BLACK = (0, 0, 0)
        self.GRAY = (200, 200, 200)
        self.BLUE = (0, 0, 255)
        self.RED = (255, 0, 0)
        self.GREEN = (0, 255, 0)
        self.LIGHT_BLUE = (173, 216, 230)
        self.LIGHT_GREEN = (144, 238, 144)
        
        # Fonts
        self.title_font = pygame.font.SysFont('Arial', 32, bold=True)
        self.font = pygame.font.SysFont('Arial', 20)
        self.small_font = pygame.font.SysFont('Arial', 16)
        
        # Input parameters
        self.secret = ""
        self.prime_p = ""
        self.num_participants = "5"  # Default value
        self.threshold = "3"  # Default value
        
        # State variables
        self.shares = []
        self.coefficients = []
        self.reconstructed_secret = None
        self.share_generation_steps = []
        self.reconstruction_steps = []
        self.current_tab = "generation"
        self.scroll_offset_gen = 0
        self.scroll_offset_rec = 0
        
        # Input boxes
        self.input_boxes = [
            {"rect": pygame.Rect(350, 120, 200, 40), "text": "", "active": False, "label": "Secret:"},
            {"rect": pygame.Rect(350, 180, 200, 40), "text": "", "active": False, "label": "Prime p:"},
            {"rect": pygame.Rect(350, 240, 200, 40), "text": self.num_participants, "active": False, "label": "Participants (n):"},
            {"rect": pygame.Rect(350, 300, 200, 40), "text": self.threshold, "active": False, "label": "Threshold (t):"}
        ]
        
        # Buttons
        self.generate_button = pygame.Rect(350, 370, 200, 50)
        self.reconstruct_button = pygame.Rect(350, 450, 200, 50)
    
    def draw_text(self, text, font, color, x, y):
        text_surface = font.render(text, True, color)
        self.screen.blit(text_surface, (x, y))
    
    def draw_multiline_text(self, lines, font, color, x, y, line_height=25, max_lines=15):
        scroll_offset = self.scroll_offset_gen if self.current_tab == "generation" else self.scroll_offset_rec
        visible_lines = lines[scroll_offset:]
        visible_lines = visible_lines[:max_lines]
        for i, line in enumerate(visible_lines):
            self.draw_text(line, font, color, x, y + i * line_height)
    
    def handle_events(self):
        for event in pygame.event.get():
            if event.type == QUIT:
                return False
            
            if event.type == MOUSEBUTTONDOWN:
                # Check input box clicks
                for box in self.input_boxes:
                    if box["rect"].collidepoint(event.pos):
                        box["active"] = True
                    else:
                        box["active"] = False
                
                # Check generate button click
                if self.generate_button.collidepoint(event.pos):
                    try:
                        secret = int(self.input_boxes[0]["text"])
                        p = int(self.input_boxes[1]["text"])
                        n = int(self.input_boxes[2]["text"])
                        t = int(self.input_boxes[3]["text"])
                        
                        if t > n:
                            raise ValueError("Threshold cannot be greater than number of participants")
                        
                        self.shares, self.coefficients, self.share_generation_steps = generate_shares_with_steps(
                            secret, n, t, p)
                        self.reconstruction_steps = []
                        self.reconstructed_secret = None
                        self.current_tab = "generation"
                        self.scroll_offset_gen = 0
                    except ValueError as e:
                        print(f"Error: {e}")
                
                # Check reconstruct button click
                if self.reconstruct_button.collidepoint(event.pos) and self.shares:
                    try:
                        p = int(self.input_boxes[1]["text"])
                        t = int(self.input_boxes[3]["text"])
                        self.reconstructed_secret, self.reconstruction_steps = reconstruct_secret_with_steps(
                            self.shares, p, t)
                        self.current_tab = "reconstruction"
                        self.scroll_offset_rec = 0
                    except ValueError as e:
                        print(f"Error: {e}")
                
                # Check tab selection
                if pygame.Rect(50, 520, 200, 40).collidepoint(event.pos):
                    self.current_tab = "generation"
                elif pygame.Rect(300, 520, 250, 40).collidepoint(event.pos):
                    self.current_tab = "reconstruction"
            
            # Handle scrolling
            if event.type == MOUSEWHEEL:
                if self.current_tab == "generation":
                    if event.y > 0 and self.scroll_offset_gen > 0:
                        self.scroll_offset_gen -= 1
                    elif event.y < 0 and self.scroll_offset_gen < len(self.share_generation_steps) - 15:
                        self.scroll_offset_gen += 1
                else:
                    if event.y > 0 and self.scroll_offset_rec > 0:
                        self.scroll_offset_rec -= 1
                    elif event.y < 0 and self.scroll_offset_rec < len(self.reconstruction_steps) - 15:
                        self.scroll_offset_rec += 1
            
            if event.type == KEYDOWN:
                # Handle text input
                active_box = next((box for box in self.input_boxes if box["active"]), None)
                if active_box:
                    if event.key == K_RETURN:
                        active_box["active"] = False
                    elif event.key == K_BACKSPACE:
                        active_box["text"] = active_box["text"][:-1]
                    else:
                        if event.unicode.isdigit():
                            active_box["text"] += event.unicode
        
        return True
    
    def draw(self):
        self.screen.fill(self.WHITE)
        
        # Draw title
        self.draw_text("Generic Shamir's Secret Sharing", self.title_font, self.BLACK, self.WIDTH//2 - 250, 30)
        
        # Draw input boxes
        for box in self.input_boxes:
            color = self.BLUE if box["active"] else self.BLACK
            pygame.draw.rect(self.screen, self.GRAY, box["rect"])
            pygame.draw.rect(self.screen, color, box["rect"], 2)
            
            # Draw label
            self.draw_text(box["label"], self.font, self.BLACK, box["rect"].x - 120, box["rect"].y + 5)
            
            # Draw input text
            text_surface = self.font.render(box["text"], True, self.BLACK)
            self.screen.blit(text_surface, (box["rect"].x + 5, box["rect"].y + 5))
        
        # Draw buttons
        pygame.draw.rect(self.screen, self.GREEN, self.generate_button)
        self.draw_text("Generate Shares", self.font, self.BLACK, self.generate_button.x + 30, self.generate_button.y + 15)
        
        pygame.draw.rect(self.screen, self.RED, self.reconstruct_button)
        self.draw_text("Reconstruct Secret", self.font, self.BLACK, self.reconstruct_button.x + 15, self.reconstruct_button.y + 15)
        
        # Tab selection
        pygame.draw.rect(self.screen, self.LIGHT_GREEN if self.current_tab == "generation" else self.GRAY, (50, 520, 200, 40))
        pygame.draw.rect(self.screen, self.LIGHT_BLUE if self.current_tab == "reconstruction" else self.GRAY, (300, 520, 250, 40))
        self.draw_text("Generation Steps", self.font, self.BLACK, 80, 530)
        self.draw_text("Reconstruction Steps", self.font, self.BLACK, 330, 530)
        
        # Display steps based on current tab
        steps_area = pygame.Rect(50, 570, self.WIDTH - 100, 250)
        pygame.draw.rect(self.screen, self.WHITE, steps_area)
        pygame.draw.rect(self.screen, self.BLACK, steps_area, 2)
        
        if self.current_tab == "generation" and self.share_generation_steps:
            self.draw_multiline_text(self.share_generation_steps, self.small_font, self.BLACK, 
                                    steps_area.x + 10, steps_area.y + 10, 20)
            if len(self.share_generation_steps) > 15:
                self.draw_text("↑ Scroll Up ↑", self.small_font, self.BLACK, self.WIDTH//2 - 50, steps_area.y - 30)
                self.draw_text("↓ Scroll Down ↓", self.small_font, self.BLACK, self.WIDTH//2 - 50, steps_area.y + 260)
        elif self.current_tab == "reconstruction" and self.reconstruction_steps:
            self.draw_multiline_text(self.reconstruction_steps, self.small_font, self.BLACK, 
                                    steps_area.x + 10, steps_area.y + 10, 20)
            if len(self.reconstruction_steps) > 15:
                self.draw_text("↑ Scroll Up ↑", self.small_font, self.BLACK, self.WIDTH//2 - 50, steps_area.y - 30)
                self.draw_text("↓ Scroll Down ↓", self.small_font, self.BLACK, self.WIDTH//2 - 50, steps_area.y + 260)
        
        # Display shares list
        if self.shares:
            shares_text = "Generated Shares: " + ", ".join([f"({x},{y})" for x, y in self.shares])
            self.draw_text(shares_text, self.small_font, self.BLACK, 50, 490)
        
        # Display final result
        if self.reconstructed_secret is not None:
            result_y = 830
            if self.reconstructed_secret == int(self.input_boxes[0]["text"]):
                self.draw_text(f"Success! Reconstructed secret: {self.reconstructed_secret}", 
                             self.font, self.GREEN, 50, result_y)
            else:
                self.draw_text(f"Error! Reconstructed: {self.reconstructed_secret}, Original: {self.input_boxes[0]['text']}", 
                             self.font, self.RED, 50, result_y)
        
        pygame.display.flip()
    
    def run(self):
        running = True
        while running:
            running = self.handle_events()
            self.draw()
        
        pygame.quit()
        sys.exit()

if __name__ == "__main__":
    app = ShamirSecretSharingGUI()
    app.run()